In [157]:
import numpy as np
import tensorflow as tf

from keras.layers import Dense, Activation
from keras.models import Model
from keras.applications import imagenet_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy, sparse_categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix

import os
import itertools
import shutil
import random
import matplotlib.pyplot as plt

In [158]:
#organize the data from the folder into train, test and validation

#1. change directory to the sign language data set
os.chdir('../Flow_Tensor/ConvNeu_Net//Sign-Language-Digits-Dataset/')
#2. check to ensure the data we are about to setup is not already on disk
if os.path.isdir('train/0/') is False:
    os.mkdir('train')
    os.mkdir('test')
    os.mkdir('valid')

    #iterate over all the direcories (0-9) in the sign lang data set
    for i in range(0,10):
        shutil.move(f'{i}','train') #moves each of the 0-9 class dirs into our train dir
         #make new class dir inside valid, with whatever place we are at in the loop 0-9 
         #NoTE: the class directories created are empty
        os.mkdir(f'valid/{i}')
        os.mkdir(f'test/{i}')

        #sample 30 random samples from the train/i class dir
        valid_samples = random.sample(os.listdir(f'train/{i}'),30)
        for j in valid_samples:
            #for each of the valid_samples picked randomly, move them from train 
            # to valid in class i - recursively
            shutil.move(f'train/{i}/{j}', f'valid/{i}') # equivalent to unix mv command

        #sample 10 random samples from the train/i class dir
        test_samples = random.sample(os.listdir(f'train/{i}'),5)
        for k in test_samples:
            shutil.move(f'train/{i}/{k}', f'test/{i}')
os.chdir('../..')


In [159]:
#pre-process the data

#define where the train, valid, test dirs are located on disk
train_path = './ConvNeu_Net/Sign-Language-Digits-Dataset/train/'
test_path = './ConvNeu_Net/Sign-Language-Digits-Dataset/test/'
valid_path = './ConvNeu_Net/Sign-Language-Digits-Dataset/valid/'

In [160]:
#setup the directory iterators -> ImageDataGenerator.flow_from_directory()

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path,target_size=(224,224), batch_size=10)
#pre-process input scales image data from -1 to 1 not 0 to 255
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10
)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path, (224,224), batch_size=10, shuffle=False
)


Found 1662 images belonging to 10 classes.
Found 100 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


In [161]:
mobile = tf.keras.applications.mobilenet.MobileNet()
#mobile.summary()

In [162]:
#fetching the model and exclusding the last 6 layers of the mobile net
x = mobile.layers[-2].output
 #store it in a variable x
output = Dense(units=10, activation='softmax')(x)

In [163]:
#construct the reconstructed model
model = Model(inputs=mobile.input, outputs=output) #we have out original mobile net layer
# we have included our layer for the last 5/6 that we ommiteed, therefoer, we have to train 
#this new model, and this we do by, freezing some parts of the previous inorder to maintin 
#the weights

In [164]:
#experimantally came up with 26 inrder ro have a decently performing model
for layer in model.layers[:-23]: #choose the last 23, omit the first (88-23)
    layer.trainable = False #train 23rd to last layer
#model.summary()

In [165]:
#Compile the Model for Training

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [166]:
#Train the Model by calling the fit function
model.fit(train_batches,
steps_per_epoch=len(train_batches),
validation_data=valid_batches,
validation_steps=len(valid_batches),
epochs=10,
verbose=2
)

Epoch 1/10
167/167 - 67s - loss: 0.8431 - val_loss: 2.9829 - 67s/epoch - 400ms/step
Epoch 2/10
167/167 - 66s - loss: 0.1154 - val_loss: 0.2950 - 66s/epoch - 397ms/step
Epoch 3/10
167/167 - 64s - loss: 0.0581 - val_loss: 0.0686 - 64s/epoch - 382ms/step
Epoch 4/10
167/167 - 68s - loss: 0.0273 - val_loss: 0.1754 - 68s/epoch - 406ms/step
Epoch 5/10
167/167 - 53s - loss: 0.0348 - val_loss: 0.1520 - 53s/epoch - 319ms/step
Epoch 6/10
167/167 - 45s - loss: 0.0375 - val_loss: 0.2782 - 45s/epoch - 271ms/step
Epoch 7/10
167/167 - 45s - loss: 0.0831 - val_loss: 0.1967 - 45s/epoch - 269ms/step
Epoch 8/10
167/167 - 72s - loss: 0.0291 - val_loss: 0.0592 - 72s/epoch - 429ms/step
Epoch 9/10
167/167 - 54s - loss: 0.0193 - val_loss: 0.0986 - 54s/epoch - 325ms/step
Epoch 10/10
167/167 - 62s - loss: 0.0095 - val_loss: 0.0522 - 62s/epoch - 373ms/step


In [174]:
#grab classes from the unshuffled data set to set the test labels
test_labels = test_batches.classes

In [175]:
#run the prediction function
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

In [176]:
#create a confusion matrix
cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

In [177]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

        print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [178]:
cm

array([[0, 1, 0, 1, 2, 3, 1, 0, 0, 2],
       [2, 4, 2, 1, 0, 0, 0, 0, 0, 1],
       [2, 0, 2, 0, 1, 1, 2, 0, 2, 0],
       [2, 0, 1, 1, 1, 0, 2, 1, 2, 0],
       [1, 2, 0, 1, 0, 1, 2, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 2, 0, 2, 0, 1, 2, 1, 1],
       [1, 0, 0, 2, 2, 0, 1, 2, 1, 1],
       [0, 0, 2, 1, 0, 3, 0, 1, 1, 2],
       [1, 1, 0, 2, 1, 1, 0, 2, 1, 1]])

In [180]:
test_batches